<a href="https://www.kaggle.com/code/rubyham/json-agent-evaluation?scriptVersionId=283178620" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Install dependencies

In [1]:
pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.

In [2]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Configure Gemini API Key

In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: Unexpected response from the service. Response: {'errors': ['No user secrets exist for kernel id 103087403 and label GOOGLE_API_KEY.'], 'error': {'code': 5}, 'wasSuccessful': False}.


Import ADK components

In [4]:
from google.genai import types

from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


Helper functions

In [5]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers
import json
import re
from typing import Dict, Any, Optional, List


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


INTENT_PATTERNS = {
    "create": r"\b(create|make|add|open|start|book|schedule)\b",
    "update": r"\b(update|change|modify|edit|reschedule)\b",
    "delete": r"\b(delete|remove|cancel)\b",
    "retrieve": r"\b(get|show|find|lookup|search|list)\b",
    "summarise": r"\b(summarise|summarize|tl;dr|condense)\b",
}

print("✅ Helper functions defined.")

✅ Helper functions defined.


Configure Retry Options

In [6]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

Create agent

In [7]:
!adk create sequential_agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

Error: Option '--api_key' requires an argument.


Create subfolders for subagent

In [8]:
import os

base = "sequential_agent"

# Create subfolders
os.makedirs(f"{base}/subagents", exist_ok=True)

# Ensure __init__.py files exist
open(f"{base}/subagents/__init__.py", "w").close()

print("📁 Created subfolders: subagents/")


📁 Created subfolders: subagents/


JSON Prompt Agent

In [9]:
%%writefile sequential_agent/subagents/json_prompt_agent.py

from google.genai import types

from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

INSTRUCTION = """
You convert a script into a STRICT JSON object for a downstream Script Breakdown agent.

HARD RULES
- Return ONLY a JSON object. No Markdown fences, no headings, no prose, no backticks.
- Use double quotes for all keys/strings. No comments. No trailing commas.
- If you cannot infer a value, use null or [] (never omit required keys).
- Default scope.units to ["scenes","beats"] unless explicitly requested otherwise.
- If scene numbers are missing, set scope.assume_scene_numbers_if_missing to true.
- UK English for any free-text fields.

SCHEMA
{
  "schema_version": "1.1",
  "task": "script_breakdown_instruction",
  "project": {
    "title": "<string>",
    "series": "<string or null>",
    "episode": "<string or null>",
    "draft_version": "<string or null>"
  },
  "scope": {
    "units": ["scenes","beats"],
    "include": ["characters","locations","props","VFX","SFX","stunts"],
    "exclude": [],
    "assume_scene_numbers_if_missing": true
  },
  "delivery": {
    "outputs": ["scene_list","character_matrix","location_list","prop_list"],
    "format": "json",
    "granularity": "scene"
  },
  "rules": {
    "character_normalisation": "uppercase canonical names",
    "location_format": "INT or EXT then location then time",
    "prop_detection": "nouns with direct physical presence",
    "ambiguous_items": "flag with rationale",
    "spoilers": "allow"
  },
  "scenes": [
    {
      "page": <int or null>,
      "scene_number": "<string or null>",
      "slugline": "<string or null>",
      "description": "<one- or two-sentence scene description>",
      "interior_exterior": "INT|EXT|INT/EXT|null",
      "location": "<string or null>",
      "time_of_day": "DAY|NIGHT|null",
      "categories": ["VFX","SFX","Stunts","Crowds","Vehicles"],
      "characters": ["<UPPERCASE NAME>", "..."],
      "props": ["<prop>", "..."]
    }
  ],
  "notes": "<free-text guidance distilled from the script>",
  "input_excerpt_checksum": "<sha1 or placeholder>",
  "source_summary": {
    "page_count_estimate": "<int>",
    "approx_runtime_minutes": "<int or null>"
  }
}

DETECTION GUIDANCE
- TITLE: Use title page if present; otherwise derive from the first slugline; else "Untitled".
- SCENE LIST: Create one entry per scene-like unit (sluglines or obvious breaks).
- PAGE: If page numbers are not present, estimate or set null (do not invent precise numbers).
- SCENE NUMBER: Use the script’s number; if missing, set null (and rely on scope.assume_scene_numbers_if_missing=true).
- DESCRIPTION: 1–2 concise sentences summarising action/intent of the scene.
- INTERIOR/EXTERIOR: Parse from slugline (INT., EXT., or INT./EXT.); else null.
- LOCATION: From slugline; use canonical wording (e.g., "VILLA LOUNGE").
- TIME OF DAY: Map to DAY or NIGHT. If ambiguous terms (e.g., EVENING, DAWN, DUSK), choose DAY for daylight, NIGHT for darkness; if unclear, null.
- CATEGORIES: Include zero or more from the allowed list only:
  ["VFX","SFX","Stunts","Crowds","Vehicles"].
  Examples: explosions → "SFX", CG creatures → "VFX", staged fights → "Stunts", underwater shots → "Underwater".
- CHARACTERS: Uppercase canonical names (merge aliases).
- PROPS: Physical items interacted with or foregrounded (phones, envelopes, weapons, etc.).
- AMBIGUITY: If uncertain, use null and add a brief note/rationale in "notes".

RETURN ONLY THE JSON OBJECT.
""".strip()


json_prompt_agent = LlmAgent(
    name="json_prompt",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    description="Converts script text into structured JSON prompt that instructs script breakdown agent.",
    instruction=INSTRUCTION,
    tools=[],
    output_key="json_prompt",
)

print("✅ Json Prompt Agent defined.")


Writing sequential_agent/subagents/json_prompt_agent.py


Sequential Agent

In [10]:
%%writefile sequential_agent/agent.py

# sequential_agent/agent.py

from .subagents.json_prompt_agent import json_prompt_agent

# ADK (this version) expects root_agent
root_agent = json_prompt_agent

# For future-compatibility / clarity, also expose agent
agent = root_agent



Writing sequential_agent/agent.py


Systematic Evaluation

1. Create evaluation configuration

In [11]:
import json
import os

# Ensure directory exists
os.makedirs("sequential_agent", exist_ok=True)

# Unified evaluation configuration for all three agents
eval_config = {
    "agents": {
        "JsonPromptAgent": {
            "criteria": {
                "json_validity": True,                   # JSON must parse
                "schema_completeness_score": 0.90,       # 90% of fields must be present
                "scene_detection_accuracy": 0.90,        # # of detected scenes must match expected within tolerance
                "no_markdown_output": True               # No backticks or markdown wrappers
            },
            "test_cases": {
                "perfect": "jp_perfect_reality_sample",
                "edge": "jp_edge_minimal_and_messy",
                "bad": "jp_bad_non_script_text"
            }
        },

        "ScriptBreakdownAgent": {
            "criteria": {
                "tool_trajectory_avg_score": 1.0,        # Must call the right tool with the right args
                "excel_file_exists": True,               # Must produce a file
                "excel_format_score": 0.95,              # Should match expected sheet structure
                "normalised_cell_values": True           # No Python lists leaking into Excel
            },
            "test_cases": {
                "perfect": "sb_perfect_from_valid_json",
                "edge": "sb_edge_weird_but_valid_json",
                "bad": "sb_bad_missing_scenes"
            }
        },

        "SchedulingAgent": {
            "criteria": {
                "tool_trajectory_avg_score": 1.0,        # Must call generate_production_schedule_doc
                "template_completeness": 0.95,           # Should contain 95% of expected headers
                "docx_created": True,                    # File exists
                "hallucination_score": 0.05              # No more than 5% invented facts
            },
            "test_cases": {
                "perfect": "sched_perfect_full_schedule",
                "edge": "sched_edge_sparse_info",
                "bad": "sched_bad_missing_or_invalid_breakdown"
            }
        }
    },

    "global_criteria": {
        "response_match_score": 0.80,                    # 80% text similarity for expected responses
        "latency_threshold_seconds": 10,                 # Max allowed latency per test
    }
}

# Save output
with open("sequential_agent/test_config.json", "w") as f:
    json.dump(eval_config, f, indent=2)

print("✅ Agent Evaluation Configuration Saved: sequential_agent/test_config.json\n")
print("📊 Includes criteria for:")
print("• JsonPromptAgent")
print("\n🎯 Tests included: perfect, edge, and bad cases")
print("📁 Ready for systematic pipeline evaluation.")


✅ Agent Evaluation Configuration Saved: sequential_agent/test_config.json

📊 Includes criteria for:
• JsonPromptAgent

🎯 Tests included: perfect, edge, and bad cases
📁 Ready for systematic pipeline evaluation.


Test cases for JSON agent

In [12]:
test_cases = {
    "eval_set_id": "script_pipeline_integration_suite",
    "eval_cases": [
        {
            "eval_id": "jp_perfect_reality_sample",
            "conversation": [
                {
                    "user_content": {
                        "parts": [
                            {
                                "text": (
                                    "<SCRIPT>\n"
                                    "Title: The Game Within\n"
                                    "Genre: Reality Drama / Competition\n"
                                    "Episode: 01 - \"First Impressions\"\n"
                                    "Page: 1\n"
                                    "Logline: Twelve strangers enter a luxury villa to compete for fame, fortune, and trust. But when\n"
                                    "alliances form and secrets leak, even friendship becomes a weapon.\n"
                                    "SCENE 1 - ARRIVAL (EXT. LUXURY VILLA - DAY)\n"
                                    "CAMERA: Drone sweeping across the ocean, revealing a sleek modern villa perched on a cliff.\n"
                                    "Tropical music fades under VO.\n"
                                    "NARRATOR (V.O.): Twelve strangers. One villa. And a game where every friendship could be your\n"
                                    "downfall.\n"
                                    "CUT TO:\n"
                                    "One by one, contestants arrive. Each greeted by the HOST, charismatic and slightly mischievous.\n"
                                    "HOST: (smiling to camera) Welcome to The Game Within. Over the next six weeks, these players\n"
                                    "will compete for one hundred thousand dollars... and something far more valuable: trust.\n"
                                    "MONTAGE: Quick cuts of contestants laughing, shaking hands, sizing each other up.\n"
                                    "CONTESTANT 1 (JENNA): I’m not here to make friends... unless it helps me win.\n"
                                    "CONTESTANT 2 (MARCUS): People underestimate nice guys. That’s their first mistake.\n"
                                    "SCENE 2 - THE FIRST CHALLENGE (INT. VILLA LOUNGE - AFTERNOON)\n"
                                    "SET: Contestants gathered in a circle of plush chairs. A large screen flickers to life.\n"
                                    "ON SCREEN: The word TRUST flashes in bold red.\n"
                                    "HOST (V.O.): Your first challenge is simple: tell a lie... and make everyone believe it’s true.\n"
                                    "REACTIONS: Shocked laughter. Whispers. Nervous glances.\n"
                                    "CONTESTANT 3 (LUCAS): Wait... so we start by lying?\n"
                                    "HOST: (appearing via screen) Exactly. The player who convinces the most people wins immunity\n"
                                    "tonight.\n"
                                    "CUT TO CONFESSIONALS:\n"
                                    "JENNA (to camera): I’m a trained actress. This is my arena.\n"
                                    "MARCUS: I hate lying. But losing? I hate that more.\n"
                                    "SCENE 3 - NIGHTFALL / STRATEGY (EXT. POOL AREA - NIGHT)\n"
                                    "CAMERA: Handheld shots. Dimly lit with fairy lights and reflections from the pool.\n"
                                    "Small groups form, whispers float over cocktails.\n"
                                    "LUCAS: (quietly to AMY) You think Jenna’s bluffing? She’s too confident.\n"
                                    "AMY: Confidence is her weapon. You should be more worried about Marcus — he’s pretending to\n"
                                    "be naive.\n"
                                    "CUT TO: MARCUS sits alone, gazing at the water. The sound of waves builds as we hear his\n"
                                    "voiceover.\n"
                                    "MARCUS (V.O.): If they think I’m harmless, I’ve already won.\n"
                                    "FADE OUT.\n"
                                    "SCENE 4 - ELIMINATION ROOM (INT. CEREMONIAL CHAMBER - NIGHT)\n"
                                    "SET DESIGN: A glowing round table. Ambient blue lights. Contestants’ faces tense.\n"
                                    "HOST: Tonight, one of you will leave the villa. Remember — in The Game Within, lies have\n"
                                    "consequences.\n"
                                    "CUT TO: Contestants exchange suspicious looks.\n"
                                    "HOST (CONT'D): The player with the fewest votes of trust... is LUCAS.\n"
                                    "LUCUS: (shocked) You’ve got to be kidding.\n"
                                    "JENNA (V.O.): One down. Eleven to go.\n"
                                    "FADE OUT.\n"
                                    "SCENE 5 - TEASER (EXT. VILLA - DAWN)\n"
                                    "MONTAGE: Quick flashes — tears, confrontation, a hidden envelope.\n"
                                    "NARRATOR (V.O.): Next time on The Game Within... the truth gets personal.\n"
                                    "FADE TO BLACK.\n"
                                    "END OF SAMPLE SCRIPT\n"
                                    "</SCRIPT>\n"
                                )
                            }
                        ]
                    },
                    # NOTE: key must be "final_response", and text should be a STRING, not a JSON object
                    "final_response": {
                        "parts": [
                            {
                                "text": (
                                    """
                            {
                              "schema_version": "1.1",
                              "task": "script_breakdown_instruction",
                              "project": {
                                "title": "The Game Within",
                                "series": "The Game Within",
                                "episode": "01 - \"First Impressions\"",
                                "draft_version": null
                              },
                              "scope": {
                                "units": ["scenes","beats"],
                                "include": ["characters","locations","props","VFX","SFX","stunts"],
                                "exclude": [],
                                "assume_scene_numbers_if_missing": true
                              },
                              "delivery": {
                                "outputs": ["scene_list","character_matrix","location_list","prop_list"],
                                "format": "json",
                                "granularity": "scene"
                              },
                              "rules": {
                                "character_normalisation": "uppercase canonical names",
                                "location_format": "INT or EXT then location then time",
                                "prop_detection": "nouns with direct physical presence",
                                "ambiguous_items": "flag with rationale",
                                "spoilers": "allow"
                              },
                              "scenes": [
                                {
                                  "page": 1,
                                  "scene_number": "SCENE 1",
                                  "slugline": "EXT. LUXURY VILLA - DAY",
                                  "description": "A drone shot reveals a modern luxury villa as the Narrator introduces the game's premise. Contestants arrive individually, greeted by the Host, and reveal their initial competitive mindsets.",
                                  "interior_exterior": "EXT",
                              "location": "LUXURY VILLA",
                              "time_of_day": "DAY",
                              "categories": ["VFX","SFX","Crowds"],
                              "characters": ["NARRATOR","HOST","JENNA","MARCUS"],
                              "props": []
                            },
                            {
                              "page": 2,
                              "scene_number": "SCENE 2",
                              "slugline": "INT. VILLA LOUNGE - AFTERNOON",
                              "description": "Contestants gather in the lounge where the Host, via a screen, reveals their first challenge: tell a convincing lie for immunity. Participants react with surprise and share their strategies in confessionals.",
                              "interior_exterior": "INT",
                              "location": "VILLA LOUNGE",
                              "time_of_day": "DAY",
                              "categories": ["Crowds"],
                              "characters": ["HOST","LUCAS","JENNA","MARCUS"],
                              "props": ["large screen"]
                            },
                            {
                              "page": 3,
                              "scene_number": "SCENE 3",
                              "slugline": "EXT. POOL AREA - NIGHT",
                              "description": "As night falls, contestants form small groups, whispering strategies over cocktails. Marcus sits alone, reflecting on his plan to appear harmless.",
                              "interior_exterior": "EXT",
                              "location": "POOL AREA",
                              "time_of_day": "NIGHT",
                              "categories": ["SFX"],
                              "characters": ["LUCAS","AMY","MARCUS"],
                              "props": ["cocktails"]
                            },
                            {
                              "page": 4,
                              "scene_number": "SCENE 4",
                              "slugline": "INT. CEREMONIAL CHAMBER - NIGHT",
                              "description": "Contestants await the elimination results in a tense, blue-lit chamber. The Host announces Lucas as the first player to leave the villa, much to his shock.",
                              "interior_exterior": "INT",
                              "location": "CEREMONIAL CHAMBER",
                              "time_of_day": "NIGHT",
                              "categories": ["Crowds"],
                              "characters": ["HOST","LUCAS","JENNA"],
                              "props": ["glowing round table"]
                            },
                            {
                              "page": 4,
                              "scene_number": "SCENE 5",
                              "slugline": "EXT. VILLA - DAWN",
                              "description": "A quick montage of tears, confrontation, and a hidden envelope teases future events for the next episode of The Game Within.",
                              "interior_exterior": "EXT",
                              "location": "VILLA",
                              "time_of_day": "DAY",
                              "categories": [],
                              "characters": ["NARRATOR"],
                              "props": ["hidden envelope"]
                            }
                          ],
                          "notes": "Page numbers for scenes 2-5 are estimated based on typical script pacing and content length. Time of day 'Afternoon' and 'Dawn' have been mapped to 'DAY'.",
                          "input_excerpt_checksum": "placeholder",
                          "source_summary": {
                            "page_count_estimate": 5,
                            "approx_runtime_minutes": 7
                          }
                        }   
                                    """
                                )
                            }
                        ]
                    },
                    "intermediate_data": {
                        "tool_uses": []
                    },
                }
            ],
        },
        {
            "eval_id": "jp_edge_minimal_and_messy",
            "conversation": [
                {
                    "user_content": {
                        "parts": [
                            {
                                "text": (
                                    "title: Late Night Call\n"
                                    "page 1\n\n"
                                    "ext. city street - night\n"
                                    "Rain pours as a single taxi pulls up. A WOMAN in a red coat hurries out.\n\n"
                                    "INT. APARTMENT LIVING ROOM - CONTINUOUS\n"
                                    "The room is dark apart from the glow of a laptop. A MAN sits on the sofa.\n\n"
                                    "ext rooftop - pre dawn\n"
                                    "The woman stands at the edge, watching the city wake up. Her phone finally rings."
                                )
                            }
                        ]
                    },
                    "final_response": {
                        "parts": [
                            {
                                "text": (
                                    """
                                    {
                                  "schema_version": "1.1",
                                  "task": "script_breakdown_instruction",
                                  "project": {
                                    "title": "Late Night Call",
                                    "series": null,
                                    "episode": null,
                                    "draft_version": null
                                  },
                                  "scope": {
                                    "units": ["scenes", "beats"],
                                    "include": ["characters", "locations", "props", "VFX", "SFX", "stunts"],
                                    "exclude": [],
                                    "assume_scene_numbers_if_missing": true
                                  },
                                  "delivery": {
                                    "outputs": ["scene_list", "character_matrix", "location_list", "prop_list"],
                                    "format": "json",
                                    "granularity": "scene"
                                  },
                                  "rules": {
                                    "character_normalisation": "uppercase canonical names",
                                    "location_format": "INT or EXT then location then time",
                                    "prop_detection": "nouns with direct physical presence",
                                    "ambiguous_items": "flag with rationale",
                                    "spoilers": "allow"
                                  },
                                  "scenes": [
                                    {
                                      "page": 1,
                                      "scene_number": null,
                                      "slugline": "EXT. CITY STREET - NIGHT",
                                      "description": "Rain pours on a dark city street as a woman in a red coat quickly exits a taxi.",
                                      "interior_exterior": "EXT",
                                      "location": "CITY STREET",
                                      "time_of_day": "NIGHT",
                                      "categories": [],
                                      "characters": ["WOMAN"],
                                      "props": ["taxi"]
                                    },
                                    {
                                      "page": 1,
                                      "scene_number": null,
                                      "slugline": "INT. APARTMENT LIVING ROOM - CONTINUOUS",
                                      "description": "Inside a dim apartment, a man sits on a sofa illuminated only by a laptop’s glow.",
                                      "interior_exterior": "INT",
                                      "location": "APARTMENT LIVING ROOM",
                                      "time_of_day": null,
                                      "categories": [],
                                      "characters": ["MAN"],
                                      "props": ["laptop", "sofa"]
                                    },
                                    {
                                      "page": 1,
                                      "scene_number": null,
                                      "slugline": "EXT. ROOFTOP - DAWN",
                                      "description": "A woman stands at the edge of a rooftop, watching the city wake as her phone finally rings.",
                                      "interior_exterior": "EXT",
                                      "location": "ROOFTOP",
                                      "time_of_day": "DAY",
                                      "categories": [],
                                      "characters": ["WOMAN"],
                                      "props": ["phone"]
                                    }
                                  ],
                                  "notes": "Scenes inferred from non-standard formatting and lowercase sluglines. Time-of-day 'pre dawn' mapped to DAY. CONTINUOUS treated as unknown time. Characters WOMAN and MAN added based on scene descriptions.",
                                  "input_excerpt_checksum": "placeholder",
                                  "source_summary": {
                                    "page_count_estimate": 1,
                                    "approx_runtime_minutes": null
                                  }
                                }                          
                                    """
                                )
                            }
                        ]
                    },
                    "intermediate_data": {
                        "tool_uses": []
                    },
                }
            ],
        },
              {
                  
            "eval_id": "jp_bad_non_script_text",
            "conversation": [
                {
                    "user_content": {
                        "parts": [
                            {
                                "text": (
                                    "This is a marketing blurb about a new TV series. "
                                    "It does not contain any scene headings or script formatting."
                                )
                            }
                        ]
                    },
                    "final_response": {
                        "parts": [
                            {
                                "text": (
                                    """       
                                    {
                                      "schema_version": "1.1",
                                      "task": "script_breakdown_instruction",
                                      "project": {
                                        "title": "Untitled",
                                        "series": null,
                                        "episode": null,
                                        "draft_version": null
                                      },
                                      "scope": {
                                        "units": ["scenes", "beats"],
                                        "include": ["characters", "locations", "props", "VFX", "SFX", "stunts"],
                                        "exclude": [],
                                        "assume_scene_numbers_if_missing": true
                                      },
                                      "delivery": {
                                        "outputs": ["scene_list", "character_matrix", "location_list", "prop_list"],
                                        "format": "json",
                                        "granularity": "scene"
                                      },
                                      "rules": {
                                        "character_normalisation": "uppercase canonical names",
                                        "location_format": "INT or EXT then location then time",
                                        "prop_detection": "nouns with direct physical presence",
                                        "ambiguous_items": "flag with rationale",
                                        "spoilers": "allow"
                                      },
                                      "scenes": [],
                                      "notes": "Source text appears to be a marketing blurb, not a formatted script. No reliable scene headings or sluglines were detected, so no scene list was generated.",
                                      "input_excerpt_checksum": "placeholder",
                                      "source_summary": {
                                        "page_count_estimate": 1,
                                        "approx_runtime_minutes": null
                                      }
                                    }

                                    """
                                )
                            }
                        ]
                    },
                    "intermediate_data": {
                        "tool_uses": []
                    },
                }
            ],
        },
        
    ],
}


Write the test cases to the integration.evalset.json in our agent's root directory.

In [13]:
import json
import os

# Ensure agent directory exists
os.makedirs("sequential_agent", exist_ok=True)

# -----------------------------------------
# Use the test_cases dictionary we created earlier
# -----------------------------------------

# (Paste your test_cases dict above this line)
# test_cases = { ... }  # ← we assume it already exists in the notebook

with open("sequential_agent/integration.evalset.json", "w") as f:
    json.dump(test_cases, f, indent=2)

print("✅ Evaluation test cases written to sequential_agent/integration.evalset.json")

# -----------------------------------------
# Print summary
# -----------------------------------------

print("\n🧪 Test scenarios:")
for case in test_cases["eval_cases"]:
    # Extract user message safely
    user_msg = case["conversation"][0]["user_content"]["parts"][0]["text"]
    print(f"• {case['eval_id']}: {user_msg[:80]}{'...' if len(user_msg) > 80 else ''}")

print("\n📊 Expected results by test:")
print("• jp_perfect_reality_sample: Should produce clean JSON, full schema, correct scenes.")
print("• jp_edge_minimal_and_messy: Should succeed, but with approximate scene detection.")
print("• jp_bad_non_script_text: Should gracefully output minimal JSON with notes.")


✅ Evaluation test cases written to sequential_agent/integration.evalset.json

🧪 Test scenarios:
• jp_perfect_reality_sample: <SCRIPT>
Title: The Game Within
Genre: Reality Drama / Competition
Episode: 01 -...
• jp_edge_minimal_and_messy: title: Late Night Call
page 1

ext. city street - night
Rain pours as a single t...
• jp_bad_non_script_text: This is a marketing blurb about a new TV series. It does not contain any scene h...

📊 Expected results by test:
• jp_perfect_reality_sample: Should produce clean JSON, full schema, correct scenes.
• jp_edge_minimal_and_messy: Should succeed, but with approximate scene detection.
• jp_bad_non_script_text: Should gracefully output minimal JSON with notes.


Run CLI Evaluation

In [14]:
print("🚀 Run this command to execute evaluation:")
!adk eval sequential_agent sequential_agent/integration.evalset.json --config_file_path=agent/test_config.json --print_detailed_results

🚀 Run this command to execute evaluation:
/usr/local/lib/python3.11/dist-packages/google/adk/evaluation/metric_evaluator_registry.py:90: UserWarning: [EXPERIMENTAL] MetricEvaluatorRegistry: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  metric_evaluator_registry = MetricEvaluatorRegistry()
/usr/local/lib/python3.11/dist-packages/google/adk/evaluation/local_eval_service.py:79: UserWarning: [EXPERIMENTAL] UserSimulatorProvider: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  user_simulator_provider: UserSimulatorProvider = UserSimulatorProvider(),
INFO:google_adk.google.adk.evaluation.eval_config:No config file supplied or file not found. Using default criteria.
Using evaluation criteria: criteria={'tool_trajectory_avg_score': 1.0, 'response_match_score': 0.8} user_simulator_config=None
Traceback (most